In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.optimize import fmin_cg, minimize

Load data

In [2]:
data = loadmat("ex3data1.mat")
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [3]:
# Get X
X_raw = data['X']
# Add ones to X_raw
X = np.c_[np.ones(X_raw.shape[0]).T, X_raw]
print("Shape of X: ", X.shape)
# Get y
y = data['y']

Shape of X:  (5000, 401)


Softmax function

In [4]:
def softmax(z):
    A = np.zeros(z.shape)
    e_z = np.exp(z);
    for i in range(e_z.shape[0]):
        A[i, :] = e_z[i, :] / np.sum(e_z[i, :])
    return A

def softmaxCostFunction(theta, X, y, lamda = 0):
    theta = np.reshape(theta, (10, 401))
    m = X.shape[0];
    z = X.dot(theta.T);
    a = softmax(z);
    
    Y = np.zeros((m, 10));
    for i in range(m):
        Y[i, y[i, 0] % 10] = 1;
    
    J = np.mean(-Y * np.log(a));
    J += (lamda/(2*m))*np.sum(np.square(theta[:, 1:]))
    return J;

def softmaxGradient(theta, X, y, lamda = 0):
    theta = np.reshape(theta, (10, 401))
    m = X.shape[0];
    z = X.dot(theta.T);
    a = softmax(z);
                       
    Y = np.zeros((m, 10));
    for i in range(m):
        Y[i, y[i, 0] % 10] = 1;

    grad = np.dot((a - Y).T, X) / m;
    grad[:, 1:] = grad[:, 1:] + (lamda/m) * theta[:, 1:]
    return (grad.flatten());

def softmaxStable(z):
    A = np.zeros(z.shape)
    e_z = np.zeros(z.shape)
    for i in range(e_z.shape[0]):
        e_z[i, :] = np.exp(z[i, :] - np.max(z[i, :]))
        A[i, :] = e_z[i, :] / np.sum(e_z[i, :])
    return A

def predictSoftmaxOneVsAll(all_theta, X):
    m, n = X.shape;
    # X : m * n
    # all_theta: n_label * n
    prob = softmaxStable(X.dot(all_theta.T));
    pred = np.argmax(prob, axis=1)
    return pred

def accuracy(pred,y):
    pred.shape = (pred.size,1)
    return np.mean((pred == y % 10))*100

In [5]:
init_theta = np.zeros((10, 401));
lamda = 0;
max_iters = 100;

theta = fmin_cg(softmaxCostFunction,init_theta,fprime = softmaxGradient,args=(X,y,lamda), maxiter = max_iters,disp = 1)

         Current function value: 0.011698
         Iterations: 100
         Function evaluations: 308
         Gradient evaluations: 308


In [6]:
theta.shape

(4010,)

In [7]:
theta = np.reshape(theta, (10, 401))
pre = predictSoftmaxOneVsAll(theta, X);
print("Accurary of softmax method ", accuracy(pre, y))

Accurary of softmax method  97.42
